# Electromagnetism

#### cf. Robert G. Jahn. **Physics of Electric Propulsion** (Dover Books on Physics)  Dover Publications.  May 26, 2006 

In [1]:
import sympy

In [2]:
import os, sys

Get the current directory:

In [3]:
currentdir = os.getcwd(); currentdir

'/home/topolo/PropD/Propulsion/EM'

In [4]:
sys.path.append(currentdir)
sys.path.append('/home/topolo/PropD/Propulsion') # you are going to type in manually the parent directory of the 
# current directory, i.e. the directory "above" the current directory, in absolute path form

#### Get the physical constants and unit conversions you need straight from NIST

In [5]:
import Physique

/home/topolo/PropD/Propulsion/Physique
/home/topolo/PropD/Propulsion/Physique/rawdata/


In [9]:
from Physique import FundConst
g_0pd = FundConst[ FundConst["Quantity"].str.contains("gravity") ]
g_0pd

,Quantity,Value,Uncertainty,Unit
303,standard acceleration of gravity,9.80665,None,m s^-2


In [7]:
g_0pd.Value.get_values()

array([Decimal('9.80665')], dtype=object)

electric constant, electric (vacuum) permittivity $\epsilon_0$ 

In [18]:
epsilon_0pd = FundConst[ FundConst['Quantity'].str.contains('electric constant')]
epsilon_0pd

,Quantity,Value,Uncertainty,Unit
76,electric constant,8.854187817E-12,None,F m^-1


In [24]:
1./(4. * sympy.pi.n() * epsilon_0pd.Value)


8987551787.99791

In [14]:
from Physique import conv as convDF
convDF.columns

Index([u'Toconvertfrom', u'to', u'Multiplyby'], dtype='object')

In [37]:
convDF[convDF['to'].str.contains("coulomb ")]
esu_to_Cpd = convDF[convDF['Toconvertfrom'].str.contains("statcoulomb")]
esu_to_Cpd

,Toconvertfrom,to,Multiplyby
405,statcoulomb,coulomb (C),3.335641E-10


In [43]:
convDF[convDF['Toconvertfrom'].str.contains("dyne ")]
dyne_to_Npd = convDF.loc[137,:]
dyne_to_Npd

Toconvertfrom    dyne (dyn)
to               newton (N)
Multiplyby         0.000010
Name: 137, dtype: object

In [15]:
convDF[convDF['Toconvertfrom'].str.contains("farad")]

,Toconvertfrom,to,Multiplyby
2,abfarad,farad (F),1.0E+9
141,EMU of capacitance (abfarad),farad (F),1.0E+9
149,ESU of capacitance (statfarad),farad (F),1.112650E-12
154,faraday (based on carbon 12),coulomb (C),96485.31
406,statfarad,farad (F),1.112650E-12


## Implementation, using Python `sympy`, of Electromagnetism in terms of differential geometry 

The (sympy documentation for the differential geometry module)[http://docs.sympy.org/dev/modules/diffgeom.html] is poor: there wasn't any explanation that connects that mathematics and physics to the commands - this *could* be attributed to the medium, in that it's difficult to display LaTeX with the code, but with jupyter notebooks, this could be alleviated. 

In [59]:
from sympy import symbols, sin, cos, pi, Function
from sympy.diffgeom import Differential, Manifold, Patch, CoordSystem, WedgeProduct
t, r, phi, z = symbols('t, r, phi, z')
M = Manifold('M',4)
Mchart = Patch('P',M)
Cart = CoordSystem('Cart',Mchart)
cylin = CoordSystem('cylin',Mchart)

In [60]:
cylin.connect_to(Cart,[t,r,phi,z],[t,r*cos(phi),r*sin(phi),z])
cylin.jacobian(Cart,[t,r,phi,z])

Matrix([
[1,        0,           0, 0],
[0, cos(phi), -r*sin(phi), 0],
[0, sin(phi),  r*cos(phi), 0],
[0,        0,           0, 1]])

Define the Electric Field 1-form.  

In [61]:
E_1 = Function('E_1'); E_2 = Function('E_2'); E_3 = Function('E_3')
E_1Cart = E_1(*Cart.coord_functions()); E_2Cart = E_2(*Cart.coord_functions()); E_3Cart = E_3(*Cart.coord_functions())
EformCart = E_1Cart*Cart.base_oneforms()[1]+E_2Cart*Cart.base_oneforms()[2]+E_3Cart*Cart.base_oneforms()[3]

Define the magnetic field 2-form.

In [62]:
B_12 = Function('B_12'); B_23 = Function('B_23'); B_31 = Function('B_31')
B_12Cart = B_12(*Cart.coord_functions()); B_23Cart = B_23(*Cart.coord_functions()); 
B_31Cart = B_31(*Cart.coord_functions());

In [63]:
BformCart = B_12Cart*WedgeProduct(Cart.base_oneform(1), Cart.base_oneform(2)) + B_23Cart*WedgeProduct(Cart.base_oneform(2), Cart.base_oneform(3)) + B_31Cart*WedgeProduct(Cart.base_oneform(3), Cart.base_oneform(1)) 

In [64]:
BformCart

B_12(Cart_0, Cart_1, Cart_2, Cart_3)*WedgeProduct(dCart_1, dCart_2) + B_23(Cart_0, Cart_1, Cart_2, Cart_3)*WedgeProduct(dCart_2, dCart_3) + B_31(Cart_0, Cart_1, Cart_2, Cart_3)*WedgeProduct(dCart_3, dCart_1)

In [65]:
F2formCart = BformCart + WedgeProduct(EformCart,Cart.base_oneform(0)) 

In [68]:
# dir(Differential(F2formCart))

In [46]:
sympy.sin( sympy.pi/4) == sympy.cos(sympy.pi/4)

True